**UTAH CRASHES ML**

    GROUP 1-7
The following are links to other models. Datasets used can be found within this repository.<br>
<br>*Classification:* <br>
[Decision Tree Best](https://github.com/EVELYN-RYAN/UTCrashesML/blob/main/BestDecisionTree.ipynb)<br>
[Multi Classification](https://github.com/EVELYN-RYAN/UTCrashesML/blob/main/MultipleClassification.ipynb)<br>
<br>*Regression:*<br>
 [OLS Best](https://github.com/EVELYN-RYAN/UTCrashesML/blob/main/BestRegression.ipynb)<br>
 [Proper Model](https://github.com/EVELYN-RYAN/UTCrashesML/blob/main/Regression(2_0).ipynb)<br>
 [ONNX Run](https://github.com/EVELYN-RYAN/UTCrashesML/blob/main/onnxModel.ipynb)



    

Before any work can be done on this data, we need to clean it.
> - Install Dependancies
> - Import Data Through API Connection
> - Remove Unessessary Values
> - Find and Remove Null Values
> - Cast values to Proper Datatypes
> - Bin and Break-out Datapoints
> - (OPTIONAL) - Export as .csv

**1. Install Dependancies**

In [ ]:
#!pip install sodapy
#!pip install pandas

If these libraries aren't available, run the code above

In [ ]:
import pandas as pd
from sodapy import Socrata

**2. Remove Unessessary Values**

In [ ]:

client = Socrata("opendata.utah.gov", None)
results = client.get("herb-zqda", limit=252500)
df = pd.DataFrame.from_records(results)
df


**3. Remove Unessessary Values**

Our reasoning behind dropping these values:
 - ROUTE / ROAD NAME -> Categorical sets with thousands of different values. (Too much to dummie code)
 - CRASH_ID -> An identifier (holds no evaluative value)
 - LAT & LONG -> Can be converted to numeric values but won't provide much applicable value
 - COUNTY / CITY NAME -> Although thes values are realitively small for dummie codes, we chose not to bias our data by where poeple live. In so doing, our models are built more to recommend tips for safer driving and less to classify bad drivers.

In [ ]:
df.dropna()
df.drop(columns=["route","crash_id","lat_utm_y","long_utm_x","main_road_name","county_name","city"],inplace=True)

**4. Find and Remove Null Values**

In [ ]:
#PRINTS THE COUNT NULLS BY EACH COLUMN
for col in df.columns:
    print(df[col].isnull().sum())

2
23
4
1939
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


SINCE NO COLUMNS HAD MORE THAN A SIGNIFICANT NUMBER OF NULL VALUES THERE IS NO REASON TO DROP ANY HERE
DROPNA() DIDN'T END UP DOING ANYTHING SO WE LOOK TO REMOVE NaN

In [ ]:
#ITERATE THROUGH EVERY ROW (SEE IF NULL) TO SEE WHAT ROWS CAN BE DELETE
for i , r in df.iterrows():
   if pd.isnull(r).any():
      df.drop(i,inplace = True)

In [ ]:
#VERIFY THAT NO COLUMNS HAVE NULL VALUES LEFT (SUM OF NULL VALUES)
for col in df.columns:
    print(df[col].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


**5. Cast values to Proper Datatypes** 

Next we need to take note of the stored date type of each column

In [ ]:
df.dtypes

crash_datetime                   object
milepoint                        object
crash_severity_id                object
work_zone_related                object
pedestrian_involved              object
bicyclist_involved               object
motorcycle_involved              object
improper_restraint               object
unrestrained                     object
dui                              object
intersection_related             object
wild_animal_related              object
domestic_animal_related          object
overturn_rollover                object
commercial_motor_veh_involved    object
teenage_driver_involved          object
older_driver_involved            object
night_dark_condition             object
single_vehicle                   object
distracted_driving               object
drowsy_driving                   object
roadway_departure                object
dtype: object

We see that all data points are strings (aka. object) so wee need to cast these dataypes appropriately 

In [ ]:
#THE API SENDS EVERY DATAPOINT AS A STRING. CAST DATA FOR BETTER CLEANING

#DATETIME - ONLY CRASH_DATETIME
df['crash_datetime'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%dT%H:%M:%S.%f')
#INTEGERS
df['crash_severity_id'] = df['crash_severity_id'].astype(int)
df['milepoint'] = df['milepoint'].astype(float)
#EVERYTHING ELSE
for col in df.columns: 
    if col == 'crash_severity_id' or col =='milepoint' or col =='crash_datetime':
        temp = 0
    else:
        df[col] = df[col].astype(int)
# IN THIS CASE OBJECTS are going to be treated still like bool values
df.dtypes

crash_datetime                   datetime64[ns]
milepoint                               float64
crash_severity_id                         int64
work_zone_related                         int64
pedestrian_involved                       int64
bicyclist_involved                        int64
motorcycle_involved                       int64
improper_restraint                        int64
unrestrained                              int64
dui                                       int64
intersection_related                      int64
wild_animal_related                       int64
domestic_animal_related                   int64
overturn_rollover                         int64
commercial_motor_veh_involved             int64
teenage_driver_involved                   int64
older_driver_involved                     int64
night_dark_condition                      int64
single_vehicle                            int64
distracted_driving                        int64
drowsy_driving                          

**6. Bin and Break-out Datapoints**

We understand now that milepoint is a numeric data type and that it is quantatative. We can bin to better organize the random and sparatic data.

In [ ]:
#SINCE MILEPOINT HAS MANY POINTS BETWEEN 1-6 LETS BIN
len = df.shape[0]
count = 0
#setting a milepoint position variable
p = 1
for x in range(0,len):
    if df.iat[count,p] < .5:
        df.iat[count,p] = 0
    elif df.iat[count,p] < 1:
        df.iat[count,p] = 1
    elif df.iat[count,p] < 1.5:
        df.iat[count,p] = 1.5
    elif df.iat[count,p] < 2:
        df.iat[count,p] = 2
    elif df.iat[count,p] < 2.5:
        df.iat[count,p] = 2.5
    elif df.iat[count,p] < 3.5:
        df.iat[count,p] = 3.5
    elif df.iat[count,p] < 4:
        df.iat[count,p] = 4
    elif df.iat[count,p] < 4.5:
        df.iat[count,p] = 4.5
    elif df.iat[count,p] < 5:
        df.iat[count,p] = 5
    elif df.iat[count,p] < 5.5:
        df.iat[count,p] = 5.5
    else:
        df.iat[count,p] = 6 
    count += 1      
df

,crash_datetime,milepoint,crash_severity_id,work_zone_related,pedestrian_involved,bicyclist_involved,motorcycle_involved,improper_restraint,unrestrained,dui,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,2019-02-08 10:56:00,5.0,2,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
1,2019-12-24 12:48:00,3.5,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2019-12-28 20:42:00,0.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,2019-12-07 00:46:00,0.0,3,0,0,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,0
4,2019-12-18 17:29:00,1.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252495,2019-11-21 23:17:00,0.0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
252496,2019-10-22 19:09:00,1.5,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
252497,2019-10-25 17:30:00,1.0,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
252498,2019-11-06 18:54:00,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Datetime values are fantastic because we can extract a ton more date regarding time of day, weekday, and even season to help us catch patterns in crash severity.

In [ ]:
#NEED TO IMPORT DATETIME FROM DATETIME EXPLICITELY
from datetime import datetime
#ADD THE COLUMNS WE CAN CREATE
pHour = 1
pDay = 2
pMonth = 3
df.insert(pHour,'hour',0)
df.insert(pDay,"weekday",0)
df.insert(pMonth,'month',0)

#DERIVE THE DATA FROM DATETIME (DATE, HOUR, WEEKDAY)
count = 0
for x in range(0,len):
    date = df.iat[count,0]   
    df.iat[count,pHour] = date.hour
    df.iat[count,pDay] = date.weekday()
    df.iat[count,pMonth] = date.month
    count += 1
df.drop(columns='crash_datetime',inplace=True)
df

,hour,weekday,month,milepoint,crash_severity_id,work_zone_related,pedestrian_involved,bicyclist_involved,motorcycle_involved,improper_restraint,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,10,4,2,5.0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,12,1,12,3.5,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,20,5,12,0.0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,0,5,12,0.0,3,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,17,2,12,1.0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252495,23,3,11,0.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
252496,19,1,10,1.5,2,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
252497,17,4,10,1.0,3,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
252498,18,2,11,0.0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


**(OPTIONAL) - Export as .csv**
Many of the opperations performed above are time consuming. By managing a local dataset it will be most ideal for quickly modeling with our dataset.

In [ ]:
#df.to_csv('APICleanedCrashes.csv')